In [ ]:
import pandas as pd
import json
import os

def analyze_journal_data():
    with open(r'C:\Users\ningji\Desktop\ai_policy\ai_policy\data\journalInfo.json', 'r', encoding='utf-8') as f:
        journal_data = json.load(f)
    with open(r'C:\Users\ningji\Desktop\ai_policy\ai_policy\data\manual_classification_template.json', 'r', encoding='utf-8') as f:
        domain_mapping = json.load(f)

    df = pd.DataFrame(journal_data)

    # Map category to domain using the first category available.
    def get_domain(categories):
        if not isinstance(categories, list) or not categories:
            return "Unknown"
        return domain_mapping.get(categories[0], "Unknown")

    df['domain'] = df['category'].apply(get_domain)

    # Determine OA status if citable OA content is greater than 0%.
    def is_oa(val):
        if isinstance(val, str):
            try:
                return float(val.replace('%', '')) > 0
            except ValueError:
                return False
        return False

    df['is_oa'] = df['% of Citable OA'].apply(is_oa)

    # Perform analysis
    domain_stats = df.groupby(['domain', 'has_ai_policy']).size().unstack(fill_value=0)
    oa_stats = df.groupby(['is_oa', 'has_ai_policy']).size().unstack(fill_value=0)
    publisher_stats = df.groupby(['publisher', 'has_ai_policy']).size().unstack(fill_value=0)

    # Define output directory and ensure it exists
    output_dir = r'C:\Users\ningji\Desktop\ai_policy\ai_policy\results\policy_res'
    os.makedirs(output_dir, exist_ok=True)

    # Save results to CSV
    domain_stats.to_csv(os.path.join(output_dir, 'domain_stats.csv'))
    oa_stats.to_csv(os.path.join(output_dir, 'oa_stats.csv'))
    publisher_stats.to_csv(os.path.join(output_dir, 'publisher_stats.csv'))

    print(f"Analysis complete. Results saved in {output_dir}")

if __name__ == '__main__':
    analyze_journal_data()

Analysis complete. Results saved in C:\Users\ningji\Desktop\ai_policy\ai_policy\results\policy_res


In [ ]:
import pandas as pd
import json
import numpy as np
from collections import defaultdict
import os

jcr_to_openalex_mapping = {
    'Health Sciences': [
        'ALLERGY', 'ANATOMY & MORPHOLOGY', 'ANDROLOGY', 'ANESTHESIOLOGY',
        'AUDIOLOGY & SPEECH-LANGUAGE PATHOLOGY', 'CARDIAC & CARDIOVASCULAR SYSTEMS',
        'CLINICAL', 'CLINICAL NEUROLOGY', 'CRITICAL CARE MEDICINE', 'DENTISTRY',
        'DERMATOLOGY', 'EMERGENCY MEDICINE', 'ENDOCRINOLOGY & METABOLISM',
        'GASTROENTEROLOGY & HEPATOLOGY', 'GENERAL & INTERNAL', 'GERIATRICS & GERONTOLOGY',
        'GERONTOLOGY', 'HEALTH CARE SCIENCES & SERVICES', 'HEALTH POLICY & SERVICES',
        'HEMATOLOGY', 'INFECTIOUS DISEASES', 'INTEGRATIVE & COMPLEMENTARY MEDICINE',
        'MEDICAL ETHICS', 'MEDICAL INFORMATICS', 'MEDICAL LABORATORY TECHNOLOGY',
        'MEDICINAL', 'MEDICINE', 'NEUROIMAGING', 'NUCLEAR MEDICINE & MEDICAL IMAGING',
        'NURSING', 'NUTRITION & DIETETICS', 'OBSTETRICS & GYNECOLOGY', 'ONCOLOGY',
        'OPHTHALMOLOGY', 'ORAL SURGERY & MEDICINE', 'ORTHOPEDICS', 'OTORHINOLARYNGOLOGY',
        'PATHOLOGY', 'PEDIATRICS', 'PERIPHERAL VASCULAR DISEASE', 'PRIMARY HEALTH CARE',
        'PSYCHIATRY', 'PUBLIC', 'RADIOLOGY', 'REHABILITATION', 'REPRODUCTIVE BIOLOGY',
        'RESPIRATORY SYSTEM', 'RHEUMATOLOGY', 'SUBSTANCE ABUSE', 'SURGERY',
        'TRANSPLANTATION', 'TROPICAL MEDICINE', 'UROLOGY & NEPHROLOGY', 'VETERINARY SCIENCES'
    ],
    
    'Life Sciences': [
        'AGRICULTURE', 'AGRONOMY', 'BIOCHEMICAL RESEARCH METHODS',
        'BIOCHEMISTRY & MOLECULAR BIOLOGY', 'BIODIVERSITY CONSERVATION', 'BIOLOGICAL',
        'BIOLOGY', 'BIOMATERIALS', 'BIOPHYSICS', 'BIOTECHNOLOGY & APPLIED MICROBIOLOGY',
        'CELL & TISSUE ENGINEERING', 'CELL BIOLOGY', 'DAIRY & ANIMAL SCIENCE',
        'DEVELOPMENTAL', 'DEVELOPMENTAL BIOLOGY', 'ECOLOGY', 'ENTOMOLOGY',
        'EVOLUTIONARY BIOLOGY', 'FISHERIES', 'FOOD SCIENCE & TECHNOLOGY', 'FORESTRY',
        'GENETICS & HEREDITY', 'HORTICULTURE', 'IMMUNOLOGY', 'LIMNOLOGY', 'MARINE',
        'MARINE & FRESHWATER BIOLOGY', 'MICROBIOLOGY', 'MOLECULAR & CHEMICAL',
        'MYCOLOGY', 'NEUROSCIENCES', 'ORNITHOLOGY', 'PARASITOLOGY',
        'PHARMACOLOGY & PHARMACY', 'PHYSIOLOGY', 'PLANT SCIENCES',
        'RESEARCH & EXPERIMENTAL', 'SOIL SCIENCE', 'TOXICOLOGY', 'VIROLOGY', 'ZOOLOGY'
    ],
    
    'Physical Sciences': [
        'ACOUSTICS', 'AEROSPACE', 'AGRICULTURAL ENGINEERING', 'ANALYTICAL', 'APPLIED',
        'ARTIFICIAL INTELLIGENCE', 'ASTRONOMY & ASTROPHYSICS', 'ATOMIC',
        'AUTOMATION & CONTROL SYSTEMS', 'BIOMEDICAL', 'CERAMICS',
        'CHARACTERIZATION & TESTING', 'CHEMICAL', 'CHEMISTRY', 'CIVIL',
        'COATINGS & FILMS', 'COMPOSITES', 'COMPUTER SCIENCE', 'CONDENSED MATTER',
        'CONSTRUCTION & BUILDING TECHNOLOGY', 'CRYSTALLOGRAPHY', 'CYBERNETICS',
        'ELECTRICAL & ELECTRONIC', 'ELECTROCHEMISTRY', 'ENERGY & FUELS', 'ENGINEERING',
        'ENVIRONMENTAL', 'ENVIRONMENTAL & OCCUPATIONAL HEALTH', 'ENVIRONMENTAL SCIENCES',
        'ENVIRONMENTAL STUDIES', 'EXPERIMENTAL', 'FLUIDS & PLASMAS',
        'GEOCHEMISTRY & GEOPHYSICS', 'GEOLOGICAL', 'GEOLOGY', 'GEOSCIENCES',
        'GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY', 'HARDWARE & ARCHITECTURE',
        'INDUSTRIAL', 'INFORMATION SCIENCE & LIBRARY SCIENCE', 'INFORMATION SYSTEMS',
        'INORGANIC & NUCLEAR', 'INSTRUMENTS & INSTRUMENTATION', 'MANUFACTURING',
        'MATERIALS SCIENCE', 'MATHEMATICAL', 'MATHEMATICAL & COMPUTATIONAL BIOLOGY',
        'MATHEMATICAL METHODS', 'MATHEMATICS', 'MECHANICAL', 'MECHANICS',
        'METALLURGY & METALLURGICAL ENGINEERING', 'METEOROLOGY & ATMOSPHERIC SCIENCES',
        'MICROSCOPY', 'MINERALOGY', 'MINING & MINERAL PROCESSING',
        'NANOSCIENCE & NANOTECHNOLOGY', 'NUCLEAR', 'NUCLEAR SCIENCE & TECHNOLOGY',
        'OCEANOGRAPHY', 'OPTICS', 'ORGANIC', 'PALEONTOLOGY', 'PAPER & WOOD',
        'PARTICLES & FIELDS', 'PETROLEUM', 'PHYSICAL', 'PHYSICS', 'POLYMER SCIENCE',
        'REMOTE SENSING', 'ROBOTICS', 'SOFTWARE ENGINEERING', 'SPECTROSCOPY',
        'STATISTICS & PROBABILITY', 'TELECOMMUNICATIONS', 'TEXTILES',
        'THEORY & METHODS', 'THERMODYNAMICS', 'TRANSPORTATION',
        'TRANSPORTATION SCIENCE & TECHNOLOGY', 'WATER RESOURCES'
    ],
    
    'Social Sciences': [
        'AGRICULTURAL ECONOMICS & POLICY', 'ANTHROPOLOGY', 'AREA STUDIES',
        'BEHAVIORAL SCIENCES', 'BUSINESS', 'COMMUNICATION', 'CRIMINOLOGY & PENOLOGY',
        'CULTURAL STUDIES', 'DEMOGRAPHY', 'DEVELOPMENT STUDIES', 'ECONOMICS',
        'EDUCATION', 'EDUCATION & EDUCATIONAL RESEARCH', 'EDUCATIONAL', 'ERGONOMICS',
        'ETHICS', 'ETHNIC STUDIES', 'FAMILY STUDIES', 'FINANCE', 'GEOGRAPHY',
        'HISTORY', 'HISTORY & PHILOSOPHY OF SCIENCE', 'HISTORY OF SOCIAL SCIENCES',
        'HOSPITALITY', 'INDUSTRIAL RELATIONS & LABOR', 'INTERNATIONAL RELATIONS',
        'LAW', 'LEGAL', 'LEISURE', 'LINGUISTICS', 'LOGIC', 'MANAGEMENT',
        'OPERATIONS RESEARCH & MANAGEMENT SCIENCE', 'POLITICAL SCIENCE',
        'PSYCHOANALYSIS', 'PSYCHOLOGY', 'PUBLIC ADMINISTRATION',
        'REGIONAL & URBAN PLANNING', 'SOCIAL', 'SOCIAL ISSUES', 'SOCIAL SCIENCES',
        'SOCIAL WORK', 'SOCIOLOGY', 'SPORT & TOURISM', 'SPORT SCIENCES',
        'URBAN STUDIES', 'WOMENS STUDIES'
    ],
    
    'Multidisciplinary': [
        'INTERDISCIPLINARY', 'INTERDISCIPLINARY APPLICATIONS', 'MULTIDISCIPLINARY',
        'MULTIDISCIPLINARY SCIENCES'
    ]
}

def get_domain_from_categories(category_string):
    if pd.isna(category_string):
        return 'Unknown'
    
    categories = [cat.strip() for cat in str(category_string).split(',')]
    
    category_to_domain = {}
    for domain, cats in jcr_to_openalex_mapping.items():
        for cat in cats:
            category_to_domain[cat] = domain
    
    domain_counts = defaultdict(int)
    total_matched = 0
    
    for category in categories:
        if category in category_to_domain:
            domain_counts[category_to_domain[category]] += 1
            total_matched += 1
    
    if total_matched == 0:
        return 'Unknown'
    
    return max(domain_counts.items(), key=lambda x: x[1])[0]

def clean_percentage(pct_str):

    if pd.isna(pct_str) or pct_str == '':
        return np.nan

    return float(str(pct_str).replace('%', ''))

def normalize_journal_name(name):

    if pd.isna(name):
        return ''
    return str(name).strip().upper()

def main():

    
    journal_df = pd.read_csv(r"C:\Users\ningji\Desktop\ai_policy\ai_policy\data\Journal.csv")
    
    journal_df['Domain'] = journal_df['Category'].apply(get_domain_from_categories)
    
    journal_df['OA_Percentage'] = journal_df['% of Citable OA'].apply(clean_percentage)
    
    journal_df['Journal_Name_Normalized'] = journal_df['Journal name'].apply(normalize_journal_name)
    
    with open(r"C:\Users\ningji\Desktop\ai_policy\ai_policy\data\journalInfo_updated.json", 'r', encoding='utf-8') as f:
        journal_info = json.load(f)
    
    name_to_domain = {}
    name_to_oa = {}
    
    for _, row in journal_df.iterrows():
        norm_name = row['Journal_Name_Normalized']
        if norm_name:
            name_to_domain[norm_name] = row['Domain']
            name_to_oa[norm_name] = row['OA_Percentage']
    
    matched_count = 0
    total_count = len(journal_info)
    
    for journal in journal_info:
        if 'journal_name' in journal:
            norm_name = normalize_journal_name(journal['journal_name'])
            if norm_name in name_to_domain:
                journal['domain'] = name_to_domain[norm_name]
                journal['oa_percentage'] = name_to_oa[norm_name]
                matched_count += 1
            else:
                journal['domain'] = 'Unknown'
                journal['oa_percentage'] = np.nan
        else:
            journal['domain'] = 'Unknown'
            journal['oa_percentage'] = np.nan
    
    print(f"Successfully matched: {matched_count}/{total_count} ({matched_count/total_count*100:.1f}%)")
    
    with open(r"C:\Users\ningji\Desktop\ai_policy\ai_policy\data\journalInfo_updated.json", 'w', encoding='utf-8') as f:
        json.dump(journal_info, f, ensure_ascii=False, indent=2)

    domain_policy_stats = defaultdict(lambda: defaultdict(int))
    
    for journal in journal_info:
        domain = journal.get('domain', 'Unknown')
        policy_category = journal.get('policy_category', 'Unknown')
        domain_policy_stats[domain][policy_category] += 1
    
    domain_stats_data = []
    for domain in domain_policy_stats:
        for policy_cat in domain_policy_stats[domain]:
            domain_stats_data.append({
                'domain': domain,
                'policy_category': policy_cat,
                'journal_count': domain_policy_stats[domain][policy_cat]
            })
    
    domain_stats_df = pd.DataFrame(domain_stats_data)
    
    os.makedirs(r"C:\Users\ningji\Desktop\ai_policy\ai_policy\results\policy_res", exist_ok=True)
    
    domain_stats_df.to_csv(r"C:\Users\ningji\Desktop\ai_policy\ai_policy\results\policy_res\domain_stats.csv", 
                          index=False, encoding='utf-8')

    oa_policy_stats = defaultdict(lambda: defaultdict(int))
    
    for journal in journal_info:
        oa_pct = journal.get('oa_percentage')
        policy_category = journal.get('policy_category', 'Unknown')
        
        if pd.isna(oa_pct):
            oa_group = 'Unknown'
        elif oa_pct >= 50:
            oa_group = 'High OA (>=50%)'
        else:
            oa_group = 'Low OA (<50%)'
        
        oa_policy_stats[oa_group][policy_category] += 1
    
    oa_stats_data = []
    for oa_group in oa_policy_stats:
        for policy_cat in oa_policy_stats[oa_group]:
            oa_stats_data.append({
                'oa_group': oa_group,
                'policy_category': policy_cat,
                'journal_count': oa_policy_stats[oa_group][policy_cat]
            })
    
    oa_stats_df = pd.DataFrame(oa_stats_data)
    oa_stats_df.to_csv(r"C:\Users\ningji\Desktop\ai_policy\ai_policy\results\policy_res\oa_stats.csv", 
                       index=False, encoding='utf-8')

    domain_counts = domain_stats_df.groupby('domain')['journal_count'].sum().sort_values(ascending=False)
    for domain, count in domain_counts.items():
        print(f"  {domain}: {count}")
    
    policy_counts = domain_stats_df.groupby('policy_category')['journal_count'].sum().sort_values(ascending=False)
    for policy, count in policy_counts.items():
        print(f"  {policy}: {count}")
    
    oa_counts = oa_stats_df.groupby('oa_group')['journal_count'].sum().sort_values(ascending=False)
    for oa_group, count in oa_counts.items():
        print(f"  {oa_group}: {count}")

if __name__ == "__main__":
    main()

匹配成功: 5114/5114 (100.0%)
  Social Sciences: 1600
  Physical Sciences: 1449
  Health Sciences: 1222
  Life Sciences: 808
  Multidisciplinary: 35
  Disclosure Required: 3556
  Not Mentioned: 1528
  Strict Prohibition: 27
  Open: 3
  Low OA (<50%): 3601
  High OA (>=50%): 1513


In [4]:
import pandas as pd
import json
import os

def analyze_normalized_publishers():
    # Normalization rules provided by the user
    publisher_mapping = {
      "AB IMPERIO INC": "AB IMPERIO INC",
      "ACAD CONFERENCES & PUBL INT LTD": "ACAD CONFERENCES & PUBL INT LTD",
      "ACAD MANAGEMENT": "Academy of Management",
      "ACAD MEDICINE SINGAPORE": "Academy of Medicine, Singapore",
      "ACADEMIC & SCIENTIFIC PUBLISHERS-ASP": "Academic & Scientific Publishers (ASP)",
      "ACADEMIC PRESS INC ELSEVIER SCIENCE": "Elsevier",
      "ACADEMIC PRESS LTD- ELSEVIER SCIENCE LTD": "Elsevier",
      "ACOUSTICAL SOC AMER AMER INST PHYSICS": "Acoustical Society of America",
      "ACTA DERMATO-VENEREOLOGICA": "Acta Dermato-Venereologica",
      "ADAM MICKIEWICZ UNIV, KALISZ": "Adam Mickiewicz University Press",
      "ADIS INT LTD": "Springer Nature",
      "AGH UNIV SCIENCE & TECHNOLOGY PRESS": "AGH University of Science and Technology Press",
      "AIP Publishing": "AIP Publishing",
      "AKADEMIAI KIADO ZRT": "Akademiai Kiado",
      "ALEXANDER VON HUMBOLDT INST INTERNET & SOC": "Alexander von Humboldt Institute for Internet and Society",
      "ALLIANCE DIAMOND OPEN ACCESS JOURNALS": "Alliance Diamond Open Access Journals",
      "AME PUBLISHING COMPANY": "AME Publishing Company",
      "AMER ACAD FAMILY PHYSICIANS": "American Academy of Family Physicians",
      "AMER ACAD PEDIATRICS": "American Academy of Pediatrics",
      "AMER ACAD PSYCHIATRY & LAW": "American Academy of Psychiatry and the Law",
      "AMER ACAD SLEEP MEDICINE": "American Academy of Sleep Medicine",
      "AMER ACCOUNTING ASSOC": "American Accounting Association",
      "AMER ASSOC ADVANCEMENT SCIENCE": "American Association for the Advancement of Science (AAAS)",
      "AMER ASSOC CANCER RESEARCH": "American Association for Cancer Research (AACR)",
      "AMER ASSOC CRITICAL CARE NURSES": "American Association of Critical-Care Nurses",
      "AMER ASSOC INTELLECTUAL and DEVELOPMENTAL DISABILITIES-AAIDD": "American Association on Intellectual and Developmental Disabilities (AAIDD)",
      "AMER ASSOC NEUROLOGICAL SURGEONS": "American Association of Neurological Surgeons",
      "AMER BOARD FAMILY MEDICINE": "American Board of Family Medicine",
      "AMER CHEMICAL SOC": "American Chemical Society (ACS)",
      "AMER COLL PHYSICIANS": "American College of Physicians",
      "AMER DENTAL ASSOC": "American Dental Association",
      "AMER DIABETES ASSOC": "American Diabetes Association",
      "AMER ECONOMIC ASSOC": "American Economic Association",
      "AMER GEOPHYSICAL UNION": "American Geophysical Union (AGU)",
      "AMER INST AERONAUTICS ASTRONAUTICS": "American Institute of Aeronautics and Astronautics (AIAA)",
      "AMER INST MATHEMATICAL SCIENCES-AIMS": "American Institute of Mathematical Sciences (AIMS)",
      "AMER MATHEMATICAL SOC": "American Mathematical Society (AMS)",
      "AMER MEDICAL ASSOC": "American Medical Association (AMA)",
      "AMER METEOROLOGICAL SOC": "American Meteorological Society (AMS)",
      "AMER MUSEUM NATURAL HISTORY": "American Museum of Natural History",
      "AMER OCCUPATIONAL THERAPY ASSOC, INC": "American Occupational Therapy Association (AOTA)",
      "AMER PHYSICAL SOC": "American Physical Society (APS)",
      "AMER PHYSIOLOGICAL SOC": "American Physiological Society",
      "AMER PHYTOPATHOLOGICAL SOC": "American Phytopathological Society (APS)",
      "AMER PSYCHIATRIC PUBLISHING, INC": "American Psychiatric Association Publishing",
      "AMER PSYCHOLOGICAL ASSOC": "American Psychological Association (APA)",
      "AMER PUBLIC HEALTH ASSOC INC": "American Public Health Association",
      "AMER ROENTGEN RAY SOC": "American Roentgen Ray Society",
      "AMER SOC CELL BIOLOGY": "American Society for Cell Biology (ASCB)",
      "AMER SOC CLINICAL INVESTIGATION INC": "American Society for Clinical Investigation (ASCI)",
      "AMER SOC ENGINEERING EDUCATION": "American Society for Engineering Education",
      "AMER SOC HEMATOLOGY": "American Society of Hematology",
      "AMER SOC MICROBIOLOGY": "American Society for Microbiology (ASM)",
      "AMER SOC NEPHROLOGY": "American Society of Nephrology (ASN)",
      "AMER SOC NEURORADIOLOGY": "American Society of Neuroradiology",
      "AMER SOC PHARMACOLOGY EXPERIMENTAL THERAPEUTICS": "American Society for Pharmacology and Experimental Therapeutics (ASPET)",
      "AMER SPEECH-LANGUAGE-HEARING ASSOC": "American Speech-Language-Hearing Association (ASHA)",
      "AMER SPINAL INJURY ASSOC": "American Spinal Injury Association",
      "AMER STATISTICAL ASSOC": "American Statistical Association (ASA)",
      "AMER THORACIC SOC": "American Thoracic Society",
      "ANNALS FAMILY MEDICINE": "Annals of Family Medicine",
      "ANNUAL REVIEWS": "Annual Reviews",
      "APPEAL PUBL SOC": "APPEAL Publishing Society",
      "ASCE-AMER SOC CIVIL ENGINEERS": "American Society of Civil Engineers (ASCE)",
      "ASIA-PACIFIC ACAD OPHTHALMOLOGY-APAO": "Asia-Pacific Academy of Ophthalmology (APAO)",
      "ASIAN-AUSTRALASIAN ASSOC ANIMAL PRODUCTION SOC": "Asian-Australasian Association of Animal Production Societies",
      "ASME": "American Society of Mechanical Engineers (ASME)",
      "ASOC ESPANOLA HISTORIA ECONOMICA": "Asociación Española de Historia Económica",
      "ASOCIACION IBEROAMERICANA EDUCACION SUPERIOR & DISTANCIA - AIESAD": "Asociación Iberoamericana de Educación Superior a Distancia (AIESAD)",
      "ASSOC BRASILEIRA PSIQUIATRIA": "Associação Brasileira de Psiquiatria",
      "ASSOC COMPUTING MACHINERY": "Association for Computing Machinery (ACM)",
      "ASSOC INFORMATION SYSTEMS": "Association for Information Systems (AIS)",
      "ASSOC RESEARCH VISION OPHTHALMOLOGY INC": "Association for Research in Vision and Ophthalmology (ARVO)",
      "ASSOC TERIOLOGICA ITALIANA": "Associazione Teriologica Italiana",
      "ASSOCIACAO BRASILEIRA PESQUISA POS-GRADUACAO FISIOTERAPIA-ABRAPG-FT": "Associação Brasileira de Pesquisa e Pós-Graduação em Fisioterapia",
      "ATHABASCA UNIV PRESS": "Athabasca University Press",
      "ATOMIC SPECTROSCOPY PRESS LTD": "Atomic Spectroscopy Press",
      "AUSTRALASIAN SOC COMPUTERS LEARNING TERTIARY EDUCATION-ASCILITE": "Australasian Society for Computers in Learning in Tertiary Education (ASCILITE)",
      "AUSTRALIAN INST CRIMINOLOGY": "Australian Institute of Criminology",
      "AUSTRALIAN NATL UNIV, DEPT HISTORY": "Australian National University",
      "AUSTRALIAN PHYSIOTHERAPY ASSOC": "Australian Physiotherapy Association",
      "Accscience Publishing": "Accscience Publishing",
      "Alpha Creation Enterprise": "Alpha Creation Enterprise",
      "BAISHIDENG PUBLISHING GROUP INC": "Baishideng Publishing Group",
      "BASTAS PUBL LTD - UK": "Bastas Publishing Ltd",
      "BENTHAM SCIENCE PUBL LTD": "Bentham Science Publishers",
      "BERGHAHN JOURNALS": "Berghahn Books",
      "BIEMDAS ACAD PUBLISHERS INC": "Biemdas Academic Publishers",
      "BIOSCIENTIFICA LTD": "BioScientifica",
      "BLOOMSBURY QATAR FOUNDATION JOURNALS": "Bloomsbury Publishing",
      "BMC": "Springer Nature",
      "BMJ PUBLISHING GROUP": "BMJ",
      "BOSTON UNIV LAW REVIEW": "Boston University School of Law",
      "BRAZILIAN SOC UROL": "Brazilian Society of Urology",
      "BRILL": "Brill",
      "BRISTOL UNIV PRESS & POLICY PRESS": "Bristol University Press",
      "BRITISH AGRICULTURAL HISTORY SOC": "British Agricultural History Society",
      "BRITISH EDITORIAL SOC BONE & JOINT SURGERY": "The British Editorial Society of Bone & Joint Surgery",
      "BRITISH INST RADIOLOGY": "British Institute of Radiology",
      "C EJ PUBLISHING GROUP": "C EJ Publishing Group",
      "CAIRO UNIV, FAC COMPUTERS & INFORMATION": "Cairo University",
      "CAMBRIDGE UNIV PRESS": "Cambridge University Press",
      "CANADIAN COMMITTEE LABOUR HISTORY": "Canadian Committee on Labour History",
      "CANADIAN OPHTHAL SOC": "Canadian Ophthalmological Society",
      "CANADIAN PSYCHOLOGICAL ASSOC": "Canadian Psychological Association",
      "CANADIAN SCIENCE PUBLISHING": "Canadian Science Publishing",
      "CAS, INST OPTICS & ELECTRONICS, ED OFF OPTO-ELECTRONIC JOURNALS": "Chinese Academy of Sciences (CAS)",
      "CELL PRESS": "Elsevier",
      "CENTERS DISEASE CONTROL & PREVENTION": "Centers for Disease Control and Prevention (CDC)",
      "CENTRO ESTUDIOS POLITICOS CONSTITUCIONALES": "Centro de Estudios Políticos y Constitucionales",
      "CEPOAT-CENTRO EST PROXIMO ORIENTE & ANTIGUEDAD TARDIA": "Centro de Estudios del Próximo Oriente y la Antigüedad Tardía",
      "CHIN SOCIETY IMMUNOLOGY": "Chinese Society for Immunology",
      "CHINA ANTI-CANCER ASSOC": "China Anti-Cancer Association",
      "CHINA ELECTRIC POWER RESEARCH INST": "China Electric Power Research Institute",
      "CHINA UNIV GEOSCIENCES, BEIJING": "China University of Geosciences (Beijing)",
      "CHINA UNIV GEOSCIENCES, WUHAN": "China University of Geosciences (Wuhan)",
      "CHINESE CHEMICAL SOC": "Chinese Chemical Society",
      "CHINESE JOURNAL CANCER RESEARCH CO": "Chinese Journal of Cancer Research",
      "CHINESE JOURNAL NATURAL MEDICINES": "Chinese Journal of Natural Medicines",
      "CHINESE LASER PRESS": "Chinese Laser Press",
      "CHINESE UNIV PRESS": "The Chinese University of Hong Kong Press",
      "CHURCHILL LIVINGSTONE": "Elsevier",
      "CHURCHILL LIVINGSTONE INC MEDICAL PUBLISHERS": "Elsevier",
      "CLEVELAND CLINIC": "Cleveland Clinic",
      "CMA-CANADIAN MEDICAL ASSOC": "Canadian Medical Association",
      "CODON PUBLICATIONS": "Codon Publications",
      "COGITATIO PRESS": "Cogitatio Press",
      "COLD SPRING HARBOR LAB PRESS, PUBLICATIONS DEPT": "Cold Spring Harbor Laboratory Press",
      "COLEGIO OFICIAL DE PSICOLOGOS DE ASTURIAS": "Colegio Oficial de Psicólogos de Asturias",
      "COLEGIO OFICIAL PSICOLOGOS MADRID": "Colegio Oficial de Psicólogos de Madrid",
      "COLL AMER PATHOLOGISTS": "College of American Pathologists",
      "COLL FAMILY PHYSICIANS CANADA": "College of Family Physicians Canada",
      "COLUMBIA JOURNAL TRANSNATIONAL LAW ASSOC": "Columbia Journal of Transnational Law",
      "COMPANY BIOLOGISTS LTD": "The Company of Biologists",
      "CONSEJO SUPERIOR INVESTIGACIONES CIENTIFICAS-CSIC": "Spanish National Research Council (CSIC)",
      "COPERNICUS GESELLSCHAFT MBH": "Copernicus Publications",
      "CORNELL LAW REVIEW": "Cornell Law Review",
      "COUNCIL FOREIGN RELAT IONS INC": "Council on Foreign Relations",
      "CRACOW UNIV ECONOMICS": "Cracow University of Economics",
      "CROATIAN SOC MEDICAL BIOCHEMISTRY & LABORATORY MEDICINE": "Croatian Society of Medical Biochemistry and Laboratory Medicine",
      "CRUSE BEREAVEMENT CARE": "Cruse Bereavement Care",
      "CSIRO PUBLISHING": "CSIRO Publishing",
      "CURRENT BIOLOGY LTD": "Elsevier",
      "CURRENT MEDICINE GROUP": "Springer Nature",
      "CZECH ACADEMY AGRICULTURAL SCIENCES": "Czech Academy of Agricultural Sciences",
      "CZECH BOTANICAL SOC": "Czech Botanical Society",
      "Cherkas Global Univ Press": "Cherkas Global University Press",
      "Chinese Center for Disease Control and Prevention": "Chinese Center for Disease Control and Prevention",
      "DE GRUYTER MOUTON": "De Gruyter",
      "DE GRUYTER POLAND SP Z O O": "De Gruyter",
      "DEUTSCHER AERZTE-VERLAG GMBH": "Deutscher Ärzte-Verlag",
      "DOVE MEDICAL PRESS LTD": "Taylor & Francis",
      "DUKE UNIV": "Duke University",
      "DUKE UNIV PRESS": "Duke University Press",
      "E H ANGLE EDUCATION RESEARCH FOUNDATION, INC": "E. H. Angle Education and Research Foundation",
      "E SCHWEIZERBARTSCHE VERLAGSBUCHHANDLUNG": "E. Schweizerbart'sche Verlagsbuchhandlung",
      "ECOLE POLYTECHNIQUE": "École Polytechnique",
      "EDICIONES DOYMA S A": "Elsevier",
      "EDICIONES PROFESIONALES INFORMACION SL-EPI": "Ediciones Profesionales de la Información",
      "EDITURA ACAD ROMANE": "Editura Academiei Române",
      "EDITURA ASE": "Bucharest University of Economic Studies",
      "EDIZIONI MINERVA MEDICA": "Edizioni Minerva Medica",
      "EDP SCIENCES S A": "EDP Sciences",
      "EDUCATIONAL PUBLISHING FOUNDATION-AMERICAN PSYCHOLOGICAL ASSOC": "American Psychological Association (APA)",
      "EDWARD ELGAR PUBLISHING LTD": "Edward Elgar Publishing",
      "ELEMENT": "Element",
      "ELSEVIER": "Elsevier",
      "ELSEVIER - DIVISION REED ELSEVIER INDIA PVT LTD": "Elsevier",
      "ELSEVIER ADVANCED TECHNOLOGY": "Elsevier",
      "ELSEVIER ESPANA": "Elsevier",
      "ELSEVIER ESPANA SLU": "Elsevier",
      "ELSEVIER FRANCE-EDITIONS SCIENTIFIQUES MEDICALES ELSEVIER": "Elsevier",
      "ELSEVIER GMBH": "Elsevier",
      "ELSEVIER INC": "Elsevier",
      "ELSEVIER IRELAND LTD": "Elsevier",
      "ELSEVIER MASSON, CORP OFF": "Elsevier",
      "ELSEVIER SCI LTD": "Elsevier",
      "ELSEVIER SCIENCE INC": "Elsevier",
      "ELSEVIER SCIENCE LONDON": "Elsevier",
      "ELSEVIER SCIENCE SA": "Elsevier",
      "ELSEVIER SINGAPORE PTE LTD": "Elsevier",
      "ELSEVIER TAIWAN": "Elsevier",
      "EMERALD GROUP PUBLISHING LTD": "Emerald Group Publishing",
      "EMW PUBLISHING": "EMW Publishing",
      "ENDOCRINE SOC": "Endocrine Society",
      "EQUINOX PUBLISHING LTD": "Equinox Publishing",
      "ESMON PUBLICIDAD S A, DEPT ALLERGY & CLIN IMMUNOL, CLIN UNIV NAVARRA": "Esmon Publicidad S.A.",
      "ETHNICITY & DISEASE, INC": "Ethnicity & Disease, Inc.",
      "EUR CENTRE DIS PREVENTION & CONTROL": "European Centre for Disease Prevention and Control (ECDC)",
      "EUROPA EDITION": "Europa Edition",
      "EUROPEAN ASSOC GEOCHEMISTRY": "European Association of Geochemistry",
      "EUROPEAN MATHEMATICAL SOC-EMS": "European Mathematical Society (EMS)",
      "EUROPEAN RESEARCH CENTER": "European Research Center",
      "EUROPEAN RESPIRATORY SOC JOURNALS LTD": "European Respiratory Society",
      "EXCERPTA MEDICA INC-ELSEVIER SCIENCE INC": "Elsevier",
      "EXCLI JOURNAL MANAGING OFFICE": "Leibniz Research Centre for Working Environment and Human Factors (IfADo)",
      "FEDERAL RESERVE BANK ST LOUIS": "Federal Reserve Bank of St. Louis",
      "FERRATA STORTI FOUNDATION": "Ferrata Storti Foundation",
      "FINNISH SOC SCIENCE & TECHNOLOGY STUDIES": "Finnish Society for Science and Technology Studies",
      "FORUM MULTIMEDIA PUBLISHING, LLC": "Forum Multimedia Publishing",
      "FOUNDATION REHABILITATION INFORMATION": "Foundation for Rehabilitation Information",
      "FRANZ STEINER VERLAG GMBH": "Franz Steiner Verlag",
      "FRENCH CENTRE RESEARCH CONTEMPORARY CHINA": "French Centre for Research on Contemporary China",
      "FRONTIERS MEDIA SA": "Frontiers",
      "FUNDACAO GETULIO VARGAS-FGV": "Fundação Getulio Vargas (FGV)",
      "FUTURE MEDICINE LTD": "Future Medicine",
      "GEBRUDER BORNTRAEGER": "Gebrüder Borntraeger",
      "GEOLOGICAL SOC AMER, INC": "Geological Society of America (GSA)",
      "GEOLOGICAL SURVEY DENMARK & GREENLAND": "Geological Survey of Denmark and Greenland",
      "GEOMETRY & TOPOLOGY PUBLICATIONS": "Geometry & Topology Publications",
      "GEORG THIEME VERLAG KG": "Thieme Medical Publishers",
      "GEORGE WASHINGTON UNIV": "George Washington University",
      "GEORGETOWN LAW JOURNAL ASSOC": "Georgetown Law Journal",
      "GEOSCIENCEWORLD": "GeoScienceWorld",
      "GESIS LEIBNIZ INST SOCIAL SCIENCES": "GESIS - Leibniz Institute for the Social Sciences",
      "GLOBAL SCIENCE PRESS": "Global Science Press",
      "GUILFORD PUBLICATIONS INC": "Guilford Press",
      "HANLEY & BELFUS-ELSEVIER INC": "Elsevier",
      "HARBORSIDE PRESS": "Harborside Press",
      "HARVARD BUSINESS SCHOOL PUBLISHING CORPORATION": "Harvard Business Publishing",
      "HARVARD GRADUATE SCHOOL EDUCATION": "Harvard Graduate School of Education",
      "HARVARD LAW REV ASSOC": "Harvard Law Review",
      "HARVARD-YENCHING INST": "Harvard-Yenching Institute",
      "HIGHER EDUCATION PRESS": "Higher Education Press",
      "HINDAWI LTD": "Wiley",
      "HISTORICAL SOC SOUTHERN PRIMORSKA KOPER-HSSP": "Historical Society of Southern Primorska",
      "HOGREFE PUBLISHING CORP": "Hogrefe Publishing Group",
      "HONG KONG ACAD MEDICINE PRESS": "Hong Kong Academy of Medicine Press",
      "HUMAN KINETICS PUBL INC": "Human Kinetics",
      "HUMANA PRESS INC": "Springer Nature",
      "Historia Sistemas Informativos -HISIN": "Historia Sistemas Informativos",
      "IEEE COMPUTER SOC": "IEEE",
      "IEEE-INST ELECTRICAL ELECTRONICS ENGINEERS INC": "IEEE",
      "IGI GLOBAL": "IGI Global",
      "IMPRINT ACADEMIC": "Imprint Academic",
      "INDERSCIENCE ENTERPRISES LTD": "Inderscience Publishers",
      "INDIANA LAW JOURNAL": "Indiana Law Journal",
      "INDIANA UNIV MATH JOURNAL": "Indiana University Mathematics Journal",
      "INDIANA UNIV PRESS": "Indiana University Press",
      "INFORMS": "INFORMS",
      "INST APPLIED MATHEMATICS": "Institute of Applied Mathematics",
      "INST BADAN GOSPODARCZYCH": "Institute for Market, Consumption and Business Cycles Research",
      "INST EARLY AMER HIST CULT": "Omohundro Institute of Early American History and Culture",
      "INST ECONOMIC RESEARCH-POLAND": "Institute for Economic Research (Poland)",
      "INST MATERIA MEDICA, CHINESE ACAD MEDICAL SCIENCES": "Chinese Academy of Medical Sciences",
      "INST MATHEMATICAL STATISTICS-IMS": "Institute of Mathematical Statistics",
      "INST MATHEMATICS & INFORMATICS": "Institute of Mathematics and Informatics",
      "INT COUNCIL RESEARCH & INNOVATION BUILDING & CONSTRUCTION": "International Council for Research and Innovation in Building and Construction (CIB)",
      "INT FORUM EDUCATIONAL TECHNOLOGY & SOC, NATL TAIWAN NORMAL UNIV": "International Forum of Educational Technology & Society",
      "INT PRESS BOSTON, INC": "International Press of Boston",
      "INT PRESS EDITING CENTRE INC": "International Press Editing Centre",
      "INT RHINOLOGIC SOC": "International Rhinologic Society",
      "INT SCIENTIFIC RESEARCH PUBLICATIONS": "International Scientific Research Publications",
      "INT SOC AGING & DISEASE": "International Society on Aging and Disease (ISOAD)",
      "INT SOC BAYESIAN ANALYSIS": "International Society for Bayesian Analysis",
      "INT SOC ENVIRON INFORM SCI": "International Society for Environmental Information Sciences",
      "INT SOC GLOBAL HEALTH": "International Society for Global Health",
      "INT UNION AGAINST TUBERCULOSIS LUNG DISEASE (I U A T L D)": "International Union Against Tuberculosis and Lung Disease",
      "INT UNION CRYSTALLOGRAPHY": "International Union of Crystallography (IUCr)",
      "INTER-RESEARCH": "Inter-Research",
      "IOP Publishing Ltd": "IOP Publishing",
      "IOS PRESS": "IOS Press",
      "IRCA-BSSA": "Seismological Society of America (SSA)",
      "ISCI-INST SCIENTIFIC COMPUTING & INFORMATION": "Institute of Scientific Computing and Information",
      "IST ITALIANO ANTROPOLOGIA": "Istituto Italiano di Antropologia",
      "IVAN FRANKO DROHOBYCH STATE PEDAGOGICAL UNIV": "Ivan Franko Drohobych State Pedagogical University",
      "IVYSPRING INT PUBL": "Ivyspring International Publisher",
      "IWA PUBLISHING": "IWA Publishing",
      "Inst Navigation - ION": "Institute of Navigation (ION)",
      "J A S S S": "Journal of Artificial Societies and Social Simulation (JASSS)",
      "J O S P T": "Journal of Orthopaedic & Sports Physical Therapy (JOSPT)",
      "JAPAN ACAD": "The Japan Academy",
      "JAPAN EPIDEMIOLOGICAL ASSOC": "Japan Epidemiological Association",
      "JAPAN NEUROSURGICAL SOC": "Japan Neurosurgical Society",
      "JAPAN PROSTHODONTIC SOC": "Japan Prosthodontic Society",
      "JAPANESE SOC ALLERGOLOGY": "Japanese Society of Allergology",
      "JAPANESE SOC HYGIENE": "Japanese Society for Hygiene",
      "JMIR PUBLICATIONS, INC": "JMIR Publications",
      "JOHN BENJAMINS PUBLISHING CO": "John Benjamins Publishing Company",
      "JOHN WILEY & SONS INC": "Wiley",
      "JOHN WILEY & SONS LTD": "Wiley",
      "JOHNS HOPKINS UNIV PRESS": "Johns Hopkins University Press",
      "JOURNAL MATER SCI TECHNOL": "Journal of Materials Science & Technology",
      "JOURNAL OF CENTRAL SOUTH UNIV": "Central South University",
      "JOURNAL STATISTICAL SOFTWARE": "Foundation for Open Access Statistics",
      "KARGER": "Karger Publishers",
      "KEAI PUBLISHING LTD": "KeAi Publishing",
      "KERMAN UNIV MEDICAL SCIENCES": "Kerman University of Medical Sciences",
      "KHAYYAM PUBL CO INC": "Khayyam Publishing Co. Inc.",
      "KLUWER LAW INT": "Wolters Kluwer",
      "KOREA HEALTH PERSONNEL LICENSING EXAMINATION INST": "Korea Health Personnel Licensing Examination Institute",
      "KOREAN ACAD MEDICAL SCIENCES": "Korean Academy of Medical Sciences",
      "KOREAN ACAD PROSTHODONTICS": "Korean Academy of Prosthodontics",
      "KOREAN ACAD REHABILITATION MEDICINE": "Korean Academy of Rehabilitation Medicine",
      "KOREAN ASSOC ORTHODONTISTS": "Korean Association of Orthodontists",
      "KOREAN ASSOC STUDY LIVER": "Korean Association for the Study of the Liver",
      "KOREAN BIOCHIP SOCIETY-KBCS": "Korean BioChip Society",
      "KOREAN DIABETES ASSOC": "Korean Diabetes Association",
      "KOREAN FIBER SOC": "Korean Fiber Society",
      "KOREAN INST METALS MATERIALS": "Korean Institute of Metals and Materials",
      "KOREAN NUCLEAR SOC": "Korean Nuclear Society",
      "KOREAN SOC ANESTHESIOLOGISTS": "Korean Society of Anesthesiologists",
      "KOREAN SOC COLOPROCTOLOGY": "Korean Society of Coloproctology",
      "KOREAN SOC GINSENG": "Korean Society of Ginseng",
      "KOREAN SOC GYNECOLOGY ONCOLOGY & COLPOSCOPY": "Korean Society of Gynecologic Oncology",
      "KOREAN SOC HORTICULTURAL SCIENCE": "Korean Society for Horticultural Science",
      "KOREAN SOC LABORATORY MEDICINE": "Korean Society for Laboratory Medicine",
      "KOREAN SOC NEPHROLOGY": "Korean Society of Nephrology",
      "KOREAN SOC OTORHINOLARYNGOL": "Korean Society of Otorhinolaryngology-Head and Neck Surgery",
      "KOREAN SOC PEDIATRIC ENDOCRINOLOGY": "Korean Society of Pediatric Endocrinology",
      "KOREAN SOC PHYCOLOGY": "Korean Society of Phycology",
      "KOREAN SOC PRECISION ENG": "Korean Society for Precision Engineering",
      "KOREAN SOC SEXUAL MEDICINE & ANDROLOGY": "Korean Society for Sexual Medicine and Andrology",
      "KOREAN SOC STUDY OBESITY": "Korean Society for the Study of Obesity",
      "KOREAN SOCIETY ANIMAL SCIENCE & TECHNOLOGY": "Korean Society of Animal Sciences and Technology",
      "KOREAN SOCIETY OF RADIOLOGY": "Korean Society of Radiology",
      "KOREAN SPINAL NEUROSURGERY SOC": "Korean Spinal Neurosurgery Society",
      "KOREAN STROKE SOC": "Korean Stroke Society",
      "Korean Pediatric Soc": "Korean Pediatric Society",
      "LABORATORY KNOWLEDGE MANAGEMENT & E-LEARNING UNIV": "Laboratory of Knowledge and Intelligent Computing",
      "LAWBOOK CO LTD": "Thomson Reuters",
      "LIFE SCIENCE ALLIANCE LLC": "Life Science Alliance",
      "LINGUISTIC SOC AMER": "Linguistic Society of America",
      "LIPPINCOTT WILLIAMS & WILKINS": "Wolters Kluwer",
      "LIVERPOOL UNIV PRESS": "Liverpool University Press",
      "MAGNOLIA PRESS": "Magnolia Press",
      "MANAGED CARE & HEALTHCARE COMMUNICATIONS LLC": "Managed Care & Healthcare Communications",
      "MARTINUS NIJHOFF PUBL": "Brill",
      "MARY ANN LIEBERT, INC": "Mary Ann Liebert, Inc.",
      "MASARYKOVA UNIV, FAC SOCIAL STUDIES": "Masaryk University",
      "MASSACHUSETTS MEDICAL SOC": "Massachusetts Medical Society",
      "MASSON EDITEUR": "Elsevier",
      "MATHEMATICAL RESEARCH PRESS-MATH RES": "Mathematical Research Press",
      "MATHEMATICAL SCIENCE PUBL": "Mathematical Sciences Publishers",
      "MDPI": "MDPI",
      "MEDICAL LIBRARY ASSOC": "Medical Library Association",
      "MEDYCYNA PRAKTYCZNA SP K SP ZOO": "Medycyna Praktyczna",
      "MICH LAW REV ASSOC": "Michigan Law Review",
      "MICHIGAN STATE UNIV PRESS": "Michigan State University Press",
      "MICROBIOLOGY SOC": "Microbiology Society",
      "MICROTOME PUBL": "Microtome Publishing",
      "MINISTRY COMMUNICATIONS & HIGH TECHNOLOGIES REPUBLIC AZERBAIJAN": "Ministry of Communications and High Technologies of the Republic of Azerbaijan",
      "MINN LAW REVIEW FOUND": "Minnesota Law Review",
      "MIT PRESS": "MIT Press",
      "MOSBY-ELSEVIER": "Elsevier",
      "MYCOSPHERE PRESS": "Mycosphere Press",
      "Medical Journal Sweden AB": "Medical Journal of Sweden",
      "NATL ACAD SCIENCES": "National Academy of Sciences (NAS)",
      "NATL ATHLETIC TRAINERS ASSOC INC": "National Athletic Trainers' Association (NATA)",
      "NATL CENTRE MARINE RESEARCH": "Hellenic Centre for Marine Research",
      "NATL COUNCIL TEACHERS MATHEMATICS-NCTM": "National Council of Teachers of Mathematics (NCTM)",
      "NATL INST ALCOHOL ABUSE ALCOHOLISM": "National Institute on Alcohol Abuse and Alcoholism (NIAAA)",
      "NATL MED ASSOC": "National Medical Association",
      "NATL MINING UNIV": "National Mining University of Ukraine",
      "NATURE PORTFOLIO": "Springer Nature",
      "NATURE PUBL GROUP": "Springer Nature",
      "ND Zelinsky Inst Organic Chemistry, RAS - ZIOC RAS": "Russian Academy of Sciences",
      "NEW LEFT REV LTD": "New Left Review",
      "NEW YORK BUSINESS GLOBAL LLC": "New York Business Global",
      "NEW YORK UNIV SCHOOL LAW": "New York University School of Law",
      "NIHR JOURNALS LIBRARY": "National Institute for Health and Care Research (NIHR)",
      "NONFERROUS METALS SOC CHINA": "The Nonferrous Metals Society of China",
      "NORTH UNIV BAIA MARE": "Technical University of Cluj-Napoca",
      "NORTHEAST FORESTRY UNIV": "Northeast Forestry University",
      "NORTHWESTERN UNIV": "Northwestern University",
      "NOW PUBLISHERS INC": "Now Publishers",
      "OAE PUBLISHING INC": "OAE Publishing",
      "OCEANOGRAPHY SOC": "The Oceanography Society",
      "OESTERREICHISCHE GESELL ENTOMOFAUNISTIK, C/O NATURHISTOR MUSEUM WIEN": "Österreichische Gesellschaft für Entomofaunistik",
      "OLD CITY PUBLISHING INC": "Old City Publishing",
      "ONLINE LEARNING CONSORTIUM": "Online Learning Consortium",
      "OTTO FRIEDRICH UNIV BAMBERG PRESS": "University of Bamberg Press",
      "OXFORD UNIV PRESS": "Oxford University Press",
      "OXFORD UNIV PRESS INC": "Oxford University Press",
      "Optica Publishing Group": "Optica Publishing Group",
      "Oxbridge Publishing House": "Oxbridge Publishing House",
      "PACIFIC AFFAIRS UNIV BRITISH COLUMBIA": "Pacific Affairs",
      "PACINI EDITORE": "Pacini Editore",
      "PAGEPRESS PUBL": "PAGEPress Publications",
      "PALGRAVE MACMILLAN LTD": "Springer Nature",
      "PEERJ INC": "PeerJ",
      "PEKING UNIV PRESS": "Peking University Press",
      "PENSOFT PUBLISHERS": "Pensoft Publishers",
      "PERGAMON-ELSEVIER SCIENCE LTD": "Elsevier",
      "PHARMACEUTICAL SOC KOREA": "Pharmaceutical Society of Korea",
      "PHYSICIANS POSTGRADUATE PRESS": "Physicians Postgraduate Press",
      "POLICY PRESS": "Bristol University Press",
      "POLISH ACAD SCIENCES INST MATHEMATICS-IMPAN": "Polish Academy of Sciences",
      "POLISH CARDIAC SOC": "Polish Cardiac Society",
      "PORTLAND PRESS LTD": "Portland Press",
      "PRESSES UNIV FRANCE": "Presses Universitaires de France",
      "PROJECT HOPE": "Project HOPE",
      "PUBLIC LIBRARY SCIENCE": "Public Library of Science (PLoS)",
      "Princeton Univ, Dept Mathematics": "Princeton University",
      "QUINTESSENCE PUBLISHING CO INC": "Quintessence Publishing",
      "R FOUNDATION STATISTICAL COMPUTING": "R Foundation for Statistical Computing",
      "RADCLIFFE CARDIOLOGY": "Radcliffe Cardiology",
      "RADICAL PHILOSOPHY": "Radical Philosophy",
      "RADIOLOGICAL SOC NORTH AMERICA (RSNA)": "Radiological Society of North America (RSNA)",
      "RAMAZAN YAMAN": "Ramazan Yaman",
      "REGIONAL EURO-ASIAN BIOLOGICAL INVASIONS CENTRE-REABIC": "Regional Euro-Asian Biological Invasions Centre (REABIC)",
      "REVIEW COMMUNICATION RESEARCH": "Review of Communication Research",
      "RIJKSHERBARIUM": "Naturalis Biodiversity Center",
      "ROCKEFELLER UNIV PRESS": "Rockefeller University Press",
      "ROUTLEDGE JOURNALS, TAYLOR & FRANCIS LTD": "Taylor & Francis",
      "ROY COLL PHYS LONDON EDITORIAL OFFICE": "Royal College of Physicians of London",
      "ROYAL COLL GENERAL PRACTITIONERS": "Royal College of General Practitioners",
      "ROYAL SOC": "Royal Society",
      "ROYAL SOC CHEMISTRY": "Royal Society of Chemistry (RSC)",
      "RUSSELL SAGE FOUNDATION": "Russell Sage Foundation",
      "Resilience Alliance": "Resilience Alliance",
      "SAGE PUBLICATIONS INC": "SAGE Publishing",
      "SAGE PUBLICATIONS LTD": "SAGE Publishing",
      "SCANDINAVIAN JOURNAL WORK ENVIRONMENT & HEALTH": "Scandinavian Journal of Work, Environment & Health",
      "SCANDINAVIAN UNIV PRESS-UNIVERSITETSFORLAGET AS": "Universitetsforlaget",
      "SCIENCE PRESS": "Science Press",
      "SCIENDO": "De Gruyter",
      "SCIPOST FOUNDATION": "SciPost Foundation",
      "SCUOLA NORMALE SUPERIORE": "Scuola Normale Superiore",
      "SEPM-SOC SEDIMENTARY GEOLOGY": "SEPM Society for Sedimentary Geology",
      "SHAHID BEHESHTI UNIV MEDICAL SCIENCES": "Shahid Beheshti University of Medical Sciences",
      "SHANGHAI JIAO TONG UNIV PRESS": "Shanghai Jiao Tong University Press",
      "SHANGHAI UNIV": "Shanghai University",
      "SHANGHAI UNIV SPORT": "Shanghai University of Sport",
      "SHENYANG PHARMACEUTICAL UNIV": "Shenyang Pharmaceutical University",
      "SIAM PUBLICATIONS": "Society for Industrial and Applied Mathematics (SIAM)",
      "SLACK INC": "SLACK Incorporated",
      "SLAVICA PUBLISHERS": "Slavica Publishers",
      "SOC BIOLGIA CHILE": "Sociedad de Biología de Chile",
      "SOC BRASIL PEDIATRIA": "Sociedade Brasileira de Pediatria",
      "SOC BRASILEIRA ICTIOLOGIA": "Sociedade Brasileira de Ictiologia",
      "SOC CULTURAL ANTHROPOLOGY": "Society for Cultural Anthropology",
      "SOC ECONOMIC GEOLOGISTS, INC": "Society of Economic Geologists",
      "SOC EXPLORATION GEOPHYSICISTS - SEG": "Society of Exploration Geophysicists (SEG)",
      "SOC INFORM MANAGE-MIS RES CENT": "Society for Information Management",
      "SOC LEARNING ANALYTICS RESEARCH-SOLAR": "Society for Learning Analytics Research (SoLAR)",
      "SOC MATHEMATIQUE FRANCE": "Société Mathématique de France",
      "SOC NEUROSCIENCE": "Society for Neuroscience",
      "SOC NUCLEAR MEDICINE INC": "Society of Nuclear Medicine and Molecular Imaging",
      "SOC PETROLEUM ENG": "Society of Petroleum Engineers (SPE)",
      "SOC SOCIOLOGICAL SCIENCE": "Society for Sociological Science",
      "SOC VAUDOISE SCIENCES NATURELLES": "Société Vaudoise des Sciences Naturelles",
      "SOUTHERN HISTORICAL ASSOC": "Southern Historical Association",
      "SPANDIDOS PUBL LTD": "Spandidos Publications",
      "SPIE-SOC PHOTO-OPTICAL INSTRUMENTATION ENGINEERS": "SPIE",
      "SPRINGER": "Springer Nature",
      "SPRINGER BASEL AG": "Springer Nature",
      "SPRINGER FRANCE": "Springer Nature",
      "SPRINGER HEIDELBERG": "Springer Nature",
      "SPRINGER INT PUBL AG": "Springer Nature",
      "SPRINGER JAPAN KK": "Springer Nature",
      "SPRINGER LONDON LTD": "Springer Nature",
      "SPRINGER SINGAPORE PTE LTD": "Springer Nature",
      "SPRINGER VIEWEG-SPRINGER FACHMEDIEN WIESBADEN GMBH": "Springer Nature",
      "SPRINGER WIEN": "Springer Nature",
      "SPRINGER-VERLAG ITALIA SRL": "Springer Nature",
      "SPRINGER/PLENUM PUBLISHERS": "Springer Nature",
      "SPRINGERNATURE": "Springer Nature",
      "STAATLICHES MUSEUM TIERKUNDE DRESDEN": "Senckenberg Natural History Collections Dresden",
      "STANFORD UNIV, STANFORD LAW SCHOOL": "Stanford Law School",
      "STATE GRID ELECTRIC POWER RESEARCH INST": "State Grid Electric Power Research Institute",
      "Shahin Digital Publisher": "Shahin Digital Publisher",
      "Soc Espanola Medicina Urgencias & Emergencias - SEMES": "Sociedad Española de Medicina de Urgencias y Emergencias (SEMES)",
      "Steklov Mathematical Inst, Russian Acad Sciences": "Russian Academy of Sciences",
      "TARU PUBLICATIONS": "Taru Publications",
      "TAYLOR & FRANCIS AS": "Taylor & Francis",
      "TAYLOR & FRANCIS AUSTRALIA": "Taylor & Francis",
      "TAYLOR & FRANCIS INC": "Taylor & Francis",
      "TAYLOR & FRANCIS LTD": "Taylor & Francis",
      "TECHNO-PRESS": "Techno-Press",
      "TERMEDIA PUBLISHING HOUSE LTD": "Termedia Publishing House",
      "TEXAS LAW REVIEW PUBL INC": "Texas Law Review",
      "THIEME MEDICAL PUBL INC": "Thieme Medical Publishers",
      "TSINGHUA UNIV PRESS": "Tsinghua University Press",
      "Tubitak Scientific & Technological Research Council Turkey": "TÜBİTAK",
      "Tuncer ACAR": "Tuncer Acar",
      "U C L A, AMER INDIAN STUDIES CENTER": "UCLA American Indian Studies Center",
      "UBIQUITY PRESS LTD": "Ubiquity Press",
      "UNIV AGRICULTURE, FAC VETERINARY SCIENCE": "University of Agriculture, Faisalabad",
      "UNIV ALICANTE, GRUPO INVESTIGACION EDUTIC-ADEI": "University of Alicante",
      "UNIV ALMERIA": "University of Almería",
      "UNIV AUTONOMA MADRID, FAC FORMACION PROFESORADO & EDUCACION": "Autonomous University of Madrid",
      "UNIV BARCELONA, DEPT HISTORIA, INST ECONOMIQUES": "University of Barcelona",
      "UNIV BELGRADE, FAC ELECTRICAL ENGINEERING": "University of Belgrade",
      "UNIV CALIF": "University of California",
      "UNIV CALIFORNIA PRESS": "University of California Press",
      "UNIV CALIFORNIA, BERKELEY SCH LAW": "University of California, Berkeley, School of Law",
      "UNIV CATOLICA NORTE, Inst Invest Arqueologicas y Museo RP Gustavo Le Paige": "Universidad Católica del Norte",
      "UNIV CHICAGO LAW SCH": "University of Chicago Law School",
      "UNIV CHICAGO PRESS": "University of Chicago Press",
      "UNIV COMPLUTENSE MADRID, SERVICIO PUBLICACIONES": "Complutense University of Madrid",
      "UNIV COSTA": "University of COSTA",
      "UNIV HAWAII PRESS": "University of Hawai'i Press",
      "UNIV HAWAII, NATL FOREIGN LANGUAGE RESOURCE CENTER": "University of Hawai'i National Foreign Language Resource Center",
      "UNIV ILLINOIS PRESS": "University of Illinois Press",
      "UNIV KRAGUJEVAC, FAC SCIENCE": "University of Kragujevac",
      "UNIV MINNESOTA PRESS": "University of Minnesota Press",
      "UNIV MURCIA": "University of Murcia",
      "UNIV NACIONAL EDUCACION DISTANCIA": "Universidad Nacional de Educación a Distancia (UNED)",
      "UNIV NAVARRA, SERVICIO PUBLICACIONES": "University of Navarra",
      "UNIV NEW SOUTH WALES, FAC LAW": "University of New South Wales",
      "UNIV NEWCASTLE": "University of Newcastle",
      "UNIV NIS": "University of Niš",
      "UNIV NORTH CAROLINA PRESS": "University of North Carolina Press",
      "UNIV PENNSYLVANIA PRESS": "University of Pennsylvania Press",
      "UNIV PITTSBURGH, UNIV LIBRARY SYSTEM": "University of Pittsburgh",
      "UNIV PRESOV": "University of Prešov",
      "UNIV SANTIAGO COMPOSTELA": "University of Santiago de Compostela",
      "UNIV SISTAN & BALUCHESTAN": "University of Sistan and Baluchestan",
      "UNIV STUDI BOLOGNA, DIPT SCIENZE STATISTICHE PAOLO FORTUNATI": "University of Bologna",
      "UNIV SZEGED, BOLYAI INSTITUTE": "University of Szeged",
      "UNIV TECHNOLOGY, SYDNEY-UTS EPRESS": "University of Technology Sydney (UTS) ePRESS",
      "UNIV TEXAS PRESS": "University of Texas Press",
      "UNIV TOMASE BATI & ZLINE, FAK MANAGEMENTU EKONOMIKY": "Tomas Bata University in Zlín",
      "UNIV TORONTO PRESS INC": "University of Toronto Press",
      "UNIV VIRGINIA LAW REVIEW ASSOC": "Virginia Law Review",
      "UNIV WINDSOR, FAC EDUCATION": "University of Windsor",
      "UNIV WISCONSIN MADISON, AFRICAN STUDIES PROGRAM": "University of Wisconsin-Madison",
      "UNIV WISCONSIN PRESS": "University of Wisconsin Press",
      "UNIV ZAGREB FAC MECHANICAL ENGINEERING & NAVAL ARCHITECTURE": "University of Zagreb",
      "UNIVERSITETSFORLAGET AS": "Universitetsforlaget",
      "US DEPT HEALTH HUMAN SCIENCES PUBLIC HEALTH SCIENCE": "U.S. Department of Health and Human Services",
      "Uspekhi Fizicheskikh Nauk": "Russian Academy of Sciences",
      "VANDERBILT LAW REVIEW": "Vanderbilt Law Review",
      "VASYL STEFANYK PRECARPATHIAN NATL UNIV": "Vasyl Stefanyk Precarpathian National University",
      "VEREIN FORDERUNG OPEN ACCESS PUBLIZIERENS QUANTENWISSENSCHAF": "Verein zur Förderung des Open Access Publizierens in den Quantenwissenschaften",
      "VIETNAM NATL UNIV": "Vietnam National University",
      "VILNIUS GEDIMINAS TECH UNIV": "Vilnius Gediminas Technical University",
      "VILNIUS UNIV, INST MATHEMATICS & INFORMATICS": "Vilnius University",
      "W B SAUNDERS CO LTD": "Elsevier",
      "W B SAUNDERS CO-ELSEVIER INC": "Elsevier",
      "W W F VERLAGSGESELLSCHAFT GMBH": "W. W. F. Verlagsgesellschaft",
      "WAGENINGEN ACADEMIC PUBLISHERS": "Wageningen Academic Publishers",
      "WALTER DE GRUYTER GMBH": "De Gruyter",
      "WESTERDIJK FUNGAL BIODIVERSITY INST": "Westerdijk Fungal Biodiversity Institute",
      "WHITE HORSE PRESS": "White Horse Press",
      "WILEY": "Wiley",
      "WILEY PERIODICALS, INC": "Wiley",
      "WILEY-HINDAWI": "Wiley",
      "WILEY-V C H VERLAG GMBH": "Wiley",
      "WOLTERS KLUWER MEDKNOW PUBLICATIONS": "Wolters Kluwer",
      "WORLD HEALTH ORGANIZATION": "World Health Organization (WHO)",
      "WORLD SCIENTIFIC PUBL CO PTE LTD": "World Scientific",
      "WROCLAW MEDICAL UNIV": "Wroclaw Medical University",
      "YALE JOURNAL ON REGULATION": "Yale Journal on Regulation",
      "YALE LAW J CO INC": "The Yale Law Journal",
      "YONSEI UNIV COLL MEDICINE": "Yonsei University College of Medicine",
      "Yandy Scientific Press": "Yandy Scientific Press",
      "ZAGREB UNIV, FAC FORESTRY": "University of Zagreb",
      "ZHEJIANG UNIV PRESS": "Zhejiang University Press",
      "eLIFE SCIENCES PUBL LTD": "eLife Sciences Publications, Ltd"
    }

    with open(r'C:\Users\ningji\Desktop\ai_policy\ai_policy\data\journalInfo_updated.json', 'r', encoding='utf-8') as f:
        journal_data = json.load(f)
    

    df = pd.DataFrame(journal_data)
    df['normalized_publisher'] = df['publisher'].map(publisher_mapping).fillna(df['publisher'])
    publisher_stats = df.groupby(['normalized_publisher', 'policy_category']).size().unstack(fill_value=0)
    output_dir = 'ai_policy/results/policy_res'
    output_path = os.path.join(output_dir, 'publisher_stats.csv')
    
    publisher_stats.to_csv(output_path, encoding='utf-8-sig')

if __name__ == '__main__':
    analyze_normalized_publishers()
    



In [ ]:
df.columns

Index(['Unnamed: 0', 'paper_id', 'date', 'has_ai_policy', 'location',
       'ai_tool', 'journal_name', 'usage_purpose', 'journal_location',
       'journal_category', 'journal_usage', 'journal_usage_categories',
       'usage_purpose_categories_filtered'],
      dtype='object')